Modified: Jul 21, 2019
# Evolution of curves using SDF

- Define a step function that encodes the discrete PDF of the curve evolution 
- Visualize the contour lines (at zero level)
- Active Contour 
    - satellite images
    - biomedical images
- Next step: 
    - agent-based clustering for image segmentation


In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, time
import numpy as np
import scipy as sp
from scipy.signal import correlate2d
import pandas as pd
    
from pathlib import Path
from pprint import pprint as pp

from sklearn.externals import joblib
import pdb

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

In [ ]:
import holoviews as hv
import xarray as xr

from holoviews import opts, dim
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.streams import Stream, param
from holoviews import streams
import geoviews as gv
import geoviews.feature as gf
from geoviews import tile_sources as gvts

import panel as pn
import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf

hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'
pn.extension()

In [ ]:
# Add the utils directory to the search path
UTILS_DIR = Path('../utils').absolute()
assert UTILS_DIR.exists()
if str(UTILS_DIR) not in sys.path:
    sys.path.insert(0, str(UTILS_DIR))
    print(f"Added {str(UTILS_DIR)} to sys.path")

# pp(sys.path)
    

In [ ]:
from utils import get_mro as mro, nprint
import utils as u
from vector import Vector as vec

### Set visualization options

In [ ]:
%opts Image [colorbar=True, active_tools=['wheel_zoom'], tools=['hover']] Curve [tools=['hover'], active_tools=['wheel_zoom']] RGB [active_tools=['wheel_zoom'], tools=['hover']]

In [ ]:
H, W = 300,300
img_opts = opts.Image(height=H, width=W, colorbar_position='bottom')
vfield_opts = opts.VectorField(width=W, height=H, color='Magnitude',
#                                magnitude=dim('Magnitude').norm()*0.2,
                               pivot='tip',
                               rescale_lengths=True)
curve_opts = opts.Points(size=5,width=W, height=H, padding=0.1, 
#                             xlim=(-10,10), ylim=(-10,10),
#                         color=dim('p')*256-50
                        )
contour_opts = opts.Contours(width=W, height=H, 
                             colorbar=False, 
                             tools=['hover'])

In [ ]:
# Grab registered bokeh renderer
print("Currently available renderers: ", *hv.Store.renderers.keys())
renderer = hv.renderer('bokeh')

In [ ]:
# wrapper to evaluate zz from sdf functions on the given x,y plane defined by `xs` and `ys`
def eval_sdf(xs, ys, sdFunc):
    zz = np.empty( (len(ys), len(xs)) )
    
    for j in range(len(ys)):
        for i in range(len(xs)):
            q = vec(xs[i],ys[j])
            zz[j,i] = sdFunc(q)
    return zz

In [ ]:
import numpy as np


In [ ]:
correl